In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
#%matplotlib widget

In [ ]:
from scipy.signal import welch, get_window
from obspy.signal.filter import bandpass

from obspy.clients.filesystem.sds import Client
from obspy.clients.fdsn import RoutingClient
from obspy.core import UTCDateTime as UTC
from obspy.signal import util

In [ ]:
import h5py

### Partial read of HDF5

In [ ]:
fname1 = 'data/GR.BFO..HHZ_2020-12-25_2020-12-31.hdf5'
fname2 = 'data/GR.BFO..HHZ_2021-01-01_2021-01-09.hdf5'

In [ ]:
fin = h5py.File(fname1, 'r')

In [ ]:
fin.__sizeof__()

In [ ]:
amps = fin['amplitudes']

In [ ]:
amps.shape

In [ ]:
l1 = [i for i in range(10)]
print(l1.__sizeof__())

In [ ]:
l2 = [i for i in range(100)]
print(l2.__sizeof__())

In [ ]:
x = 10
x.__sizeof__()

Get numpy-arrays

In [ ]:
PXX = np.load("data/GR.BFO..HHZ_2021-07-20T00:00:00.000000Z_2021-10-25T00:00:00.000000Z_PXX.npy")
AMP = np.load("data/GR.BFO..HHZ_2021-07-20T00:00:00.000000Z_2021-10-25T00:00:00.000000Z_AMP.npy")

In [ ]:
print(PXX.shape, PXX.size)
print(AMP.shape, AMP.size)

Check file sizes of numpy data

In [ ]:
%ls -lh data/*.npy

#### HPF5

In [ ]:
f = h5py.File('mmytest.hdf5', 'w')

In [ ]:
f.create_dataset('amp', data=AMP)

In [ ]:
for k, v in f.items():
    print(k, v.name)


In [ ]:
print(type(f['amp']))

To convert hdf5-dataset into numpy array, simply use `np.array()` on it.

In [ ]:
x = np.array(v)

print(type(x))

Put each numpy-array in hdf5 and check file size

In [ ]:
with h5py.File("data/GR.BFO..HHZ_PXX.hdf5", "w") as fout:
    fout.create_dataset('pxx', data=PXX, compression=1)

In [ ]:
with h5py.File("data/GR.BFO..HHZ_AMP.hdf5", "w") as fout:
    fout.create_dataset('amp', data=AMP)

In [ ]:
%ls -lh data/*.hdf5

Clear all items from FileObject

In [ ]:
f.clear()

### How to put data into HDF5 row by row?

In [ ]:
PXX.shape

In [ ]:
fname = "data/GR.BFO..HHZ_PXX.hdf5"

Initialize file with some shape (years later) and max shape (leap year).

In [ ]:
with h5py.File(fname, "w") as fout:
    fout.create_dataset('pxx', shape=(97,24,1025), maxshape=(98, 24, 1025))

In [ ]:
for i, row in enumerate(PXX):
    #print(row.shape)
    with h5py.File(fname, "a") as fout:
        fout['pxx'][i,:,:] = row

Now let's add last row again into next row.

In [ ]:
with h5py.File(fname, "a") as fout:
    try:
        fout['pxx'][i+1,:,:] = row
    except IndexError:
        fout['pxx'].resize(i+2, axis=0)
        fout['pxx'][i+1,:,:] = row

And again with increase `i`. Hopefully this does not work.

In [ ]:
i = i+1
with h5py.File(fname, "a") as fout:
    try:
        fout['pxx'][i+1,:,:] = row
    except IndexError:
        fout['pxx'].resize(98, axis=0)
        fout['pxx'][i+1,:,:] = row

No, it does not. Returns `IndexError`.

In [ ]:
fout.close()

In [ ]:
fname = "data/GR.BFO..HHZ.hdf5"

In [ ]:
with h5py.File(fname, "a") as fout:
    fout.create_dataset('PXX', data=PXX)
    fout.create_dataset('AMP', data=AMP)

### Runtime of IO

Which one is faster? 

- Open and close the file every time we add a row (simulate daily update)
- Opening it once and add all the rows one by one, then close (simulate batch processing of existing data)

Unsurprisingly, the latter is faster, although the difference is smaller than expected. Probably, looping over the array alone already costs a lot of time.

In [ ]:
%%timeit -r 20 -n 20
with h5py.File(fname, "w") as fout:
    fout.create_dataset('pxx', shape=(97,24,1025), maxshape=(98, 24, 1025))
    
for i, row in enumerate(PXX):
    #print(row.shape)
    with h5py.File(fname, "a") as fout:
        fout['pxx'][i,:,:] = row

In [ ]:
%%timeit -r 20 -n 20
with h5py.File(fname, "w") as fout:
    fout.create_dataset('pxx', shape=(97,24,1025), maxshape=(98, 24, 1025))
    
with h5py.File(fname, "a") as fout:
    for i, row in enumerate(PXX):
    #print(row.shape)
        fout['pxx'][i,:,:] = row

In [ ]:
%ls -lh data/*.hdf5

In [ ]:
with h5py.File(fname, "r") as f:
    print(f['pxx'].shape)
    dataset = f['pxx']
    data = np.array(f['pxx'])

In [ ]:
dataset.resize()

In [ ]:
%lsmagic

In [ ]:

PXX.__sizeof__()